## DenseNet

In [1]:
!nvidia-smi

Sat Mar 19 20:25:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:0B.0 Off |                    0 |
| N/A   35C    P0    43W / 300W |      0MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import paddle
import numpy as np

paddle.__version__, paddle.device.get_device()

('2.2.2', 'gpu:0')

### Data loading

In [3]:
import paddle.vision.transforms as T
from paddle.vision.datasets import FashionMNIST

# loading and normalization
# resize the shape to 96x96
transform = [T.Resize(size=96), T.Normalize(mean=[127.5], std=[127.5])]
transform = T.Compose(transform)

# constructing traning set and test set
fashionmnist_train = FashionMNIST(mode='train', transform=transform)
fashionmnist_test = FashionMNIST(mode='test', transform=transform)

In [4]:
# nums of train set and test set
len(fashionmnist_train), len(fashionmnist_test)

(60000, 10000)

In [5]:
# As for one sample, channel = 1, height, weight = 96
# for adapting to the input shape of densenet
fashionmnist_train[0][0].shape, fashionmnist_test[0][0].shape

((1, 96, 96), (1, 96, 96))

In [6]:
train_loader = paddle.io.DataLoader(fashionmnist_train, batch_size=256, shuffle=True)
test_loader = paddle.io.DataLoader(fashionmnist_test, batch_size=64, shuffle=False)

## Model

#### Dense blocks and transition layers

In [17]:
from paddle import nn
from paddle.nn import functional as F

# DenseNet uses the modified “batch normalization, activation, and convolution” structure of ResNet
# here is an implementation of this convolution block structure
 
def conv_block(input_channels, num_channels):
    return nn.Sequential(
        nn.BatchNorm2D(input_channels), nn.ReLU(),
        nn.Conv2D(input_channels, num_channels, kernel_size=3, padding=1))

In [18]:
class DenseBlock(nn.Layer):
    def __init__(self, num_convs, input_channels, num_channels):
        super(DenseBlock, self).__init__()
        layer = []
        for i in range(num_convs):
            layer.append(conv_block(num_channels * i + input_channels, num_channels))
        self.net = nn.Sequential(*layer)
        # self.cnt = 0

    def forward(self, X):
        for blk in self.net:
            # print(blk)
            # print(self.cnt)
            # self.cnt += 1
            Y = blk(X)
            # print(X.shape, Y.shape)
            # Concatenate the input and output of each block on the channel dimension
            X = paddle.concat((X, Y), axis=1)
        return X

In [19]:
blk = DenseBlock(2, 3, 10)
X = paddle.rand([4, 3, 8, 8])
Y = blk(X)
Y.shape

[4, 23, 8, 8]

In [20]:
def transition_block(input_channels, num_channels):
    return nn.Sequential(
        nn.BatchNorm2D(input_channels), nn.ReLU(),
        nn.Conv2D(input_channels, num_channels, kernel_size=1),
        nn.AvgPool2D(kernel_size=2, stride=2))

In [21]:
Y = paddle.rand([4, 23, 8, 8])
blk = transition_block(23, 10)
blk(Y).shape

[4, 10, 4, 4]

#### DenseNet Model

In [22]:
b1 = nn.Sequential(
    nn.Conv2D(1, 64, kernel_size=7, stride=2, padding=3),
    nn.BatchNorm2D(64), nn.ReLU(),
    nn.MaxPool2D(kernel_size=3, stride=2, padding=1))

In [23]:
# `num_channels`: the current number of channels
num_channels, growth_rate = 64, 32
num_convs_in_dense_blocks = [4, 4, 4, 4]
blks = []
for i, num_convs in enumerate(num_convs_in_dense_blocks):
    blks.append(DenseBlock(num_convs, num_channels, growth_rate))
    # This is the number of output channels in the previous dense block
    num_channels += num_convs * growth_rate
    # A transition layer that halves the number of channels is added between
    # the dense blocks
    if i != len(num_convs_in_dense_blocks) - 1:
        blks.append(transition_block(num_channels, num_channels // 2))
        num_channels = num_channels // 2

In [24]:
densenet = nn.Sequential(
    b1, *blks,
    nn.BatchNorm2D(num_channels), nn.ReLU(),
    nn.AdaptiveAvgPool2D((1, 1)),
    nn.Flatten(),
    nn.Linear(num_channels, 10))

In [26]:
# instantiation
model = paddle.Model(densenet)

# visualization of the model workflow
model.summary((-1, 1, 96, 96)) # [N C H W]

-------------------------------------------------------------------------------
   Layer (type)         Input Shape          Output Shape         Param #    
     Conv2D-14        [[1, 1, 96, 96]]     [1, 64, 48, 48]         3,200     
  BatchNorm2D-14     [[1, 64, 48, 48]]     [1, 64, 48, 48]          256      
      ReLU-14        [[1, 64, 48, 48]]     [1, 64, 48, 48]           0       
    MaxPool2D-1      [[1, 64, 48, 48]]     [1, 64, 24, 24]           0       
  BatchNorm2D-15     [[1, 64, 24, 24]]     [1, 64, 24, 24]          256      
      ReLU-15        [[1, 64, 24, 24]]     [1, 64, 24, 24]           0       
     Conv2D-15       [[1, 64, 24, 24]]     [1, 32, 24, 24]        18,464     
  BatchNorm2D-16     [[1, 96, 24, 24]]     [1, 96, 24, 24]          384      
      ReLU-16        [[1, 96, 24, 24]]     [1, 96, 24, 24]           0       
     Conv2D-16       [[1, 96, 24, 24]]     [1, 32, 24, 24]        27,680     
  BatchNorm2D-17     [[1, 128, 24, 24]]    [1, 128, 24, 24]   

{'total_params': 764834, 'trainable_params': 751618}

### Training

In [27]:
# optimizer and loss
model.prepare(optimizer=paddle.optimizer.Adam(parameters=model.parameters(), learning_rate=1e-3),
              loss=nn.CrossEntropyLoss(),
              metrics=paddle.metric.Accuracy())

# training
model.fit(train_loader,
        # eval_data = test_loader
        epochs=20,
        verbose=1,
        )

The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/20


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return (isinstance(seq, collections.Sequence) and
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/nn/layer/norm.py:653: UserWarning: When training, we now always track global mean and variance.
  "When training, we now always track global mean and variance.")


step 235/235 [==============================] - loss: 0.3609 - acc: 0.8555 - 72ms/step          
Epoch 2/20
step 235/235 [==============================] - loss: 0.2791 - acc: 0.9079 - 74ms/step          
Epoch 3/20
step 235/235 [==============================] - loss: 0.2342 - acc: 0.9204 - 74ms/step          
Epoch 4/20
step 235/235 [==============================] - loss: 0.3805 - acc: 0.9287 - 77ms/step          
Epoch 5/20
step 235/235 [==============================] - loss: 0.1076 - acc: 0.9327 - 75ms/step          
Epoch 6/20
step 235/235 [==============================] - loss: 0.2494 - acc: 0.9394 - 73ms/step          
Epoch 7/20
step 235/235 [==============================] - loss: 0.1635 - acc: 0.9439 - 73ms/step          
Epoch 8/20
step 235/235 [==============================] - loss: 0.1548 - acc: 0.9495 - 74ms/step          
Epoch 9/20
step 235/235 [==============================] - loss: 0.1424 - acc: 0.9531 - 75ms/step          
Epoch 10/20
step 235/235 [=============

In [28]:
model.evaluate(fashionmnist_test, batch_size=64, verbose=1)

Eval begin...
step 157/157 [==============================] - loss: 0.0100 - acc: 0.9229 - 19ms/step          
Eval samples: 10000


{'loss': [0.009955257], 'acc': 0.9229}